In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from string import punctuation
from time import sleep,time
from numpy.random import default_rng



In [145]:

strip_=lambda x: x.strip().rstrip(punctuation)

def try_css_selector(soup,css):
    x=soup.select_one(css)
    try:
        return x.text
    except AttributeError:
        return None


def details(html):
    soup=BeautifulSoup(html.content,'html.parser')

    attributes=dict()

    title=try_css_selector(soup,".book-name") 
    title2=try_css_selector(soup,".font-20.book-name")
    if title2:
        title= title +" "+title2

    attributes['title']=title
    attributes["writer"]=try_css_selector(soup,'.book-name+ .book-c a')
    attributes["translator"]=try_css_selector(soup,'.book-c+ .book-c a')  
     
    try :
        
       attributes['price_total'] = int(try_css_selector(soup,".price-slash").split()[0].replace(",",""))
    except:
        attributes['price_total'] = None

    if attributes['price_total']  is not None:
       price_discoun=int(strip_(try_css_selector(soup,".font-18")).split()[0].replace(",",""))
    else:price_discoun = None

    attributes['price_discoun']=price_discoun

    attributes['publisher']=try_css_selector(soup,".book-publisher-edition .book-publisher-edition")


    attributes['num_print'] = try_css_selector(soup,".book-publisher-edition+ .book-publisher-edition")
    attributes['category'] = try_css_selector(soup,".book-topic-link:nth-child(1)")

    try:
        page =int( try_css_selector(soup,".book-detail:nth-child(4)").split()[0])
    except :
        page=None
    attributes['page']=page

    try:
        ISBN =int( try_css_selector(soup,".mt-3+ span").split()[1])
    except :
        ISBN=None

    attributes["ISBN"]=ISBN

    try:
        weight = int(try_css_selector(soup,".book-detail:nth-child(5)").split()[0])
    except :
        weight=None
    
    attributes['weight']=weight

    try:
        credits = int(try_css_selector(soup,".book-detail:nth-child(8)").split()[1])
    except :
        credits=None
    
    attributes['credits']=credits

    return attributes



In [ ]:
list_attributes=['writer', 'translator', 'title', 'price_total', 'price_discoun', 'publisher', 'num_print', 'category', 'page', 'ISBN', 'weight', 'credits']
df=pd.DataFrame()
nan=0
t=time()
t1=time()
for i in range(1,101):
    print("i = %d"%i)
    if nan > 9: break
    
    html = requests.get("https://www.bennubook.com/book/%d" %i)
    if html.status_code == 200:
       
        
        output=pd.DataFrame(details(html),index=[0])
        
        if output.isnull().sum().sum()>10:
            nan+=1
        else: 
            nan=0
            df=pd.concat([df,output],axis=0,ignore_index=True)
        
        print("nan = %d"%nan)
        
        
    rand=  default_rng().uniform(9,23) 
    print('rand= %d' %rand) 
    sleep(rand)
    print(f'time = {time()-t}')
    t=time()
    print(f"total time = {time()-t1}")
    
    
df.to_excel("book.xlsx")

In [149]:

df.to_excel("book.xlsx")
